### This is the old AI Backend that uses FALCON-7B Model to generate MCQs and Headings.

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes safetensors
!pip install fastapi nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!nvidia-smi

Fri Mar 29 05:13:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=4096,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from langchain import PromptTemplate,  LLMChain

# Initialize FastAPI
app = FastAPI()

class ParagraphInput(BaseModel):
    paragraph: str

# Define route
@app.get("/generate-mcqs/")
async def generate_mcqs(input_data: ParagraphInput):
  template = """
  You are an intelligent chatbot. Based on the information provided in the paragraph below, generate three multiple-choice questions (MCQs) that test the user's understanding and critical thinking. Ensure that the questions challenge the user's knowledge and comprehension of the material and avoid creating trivial or overly obvious questions.

Paragraph:
{paragraph}

Instructions:
1. Each MCQ should have question statement and four answer options, including one correct answer and three distractors.
2. The correct answer should be plausible and directly related to the information presented in the paragraph.
3. Distractors should be reasonable but incorrect choices that may reflect common misconceptions or misinterpretations.
4. Ensure that the questions and answer options are clear, concise, and grammatically correct.
5. Avoid questions that can be answered simply by recalling facts from the paragraph. Instead, aim to assess deeper understanding, critical thinking, and the ability to apply knowledge.

Format for Responses:
For each multiple-choice question, provide the question text followed by four answer options, separated by line breaks. Mark the correct answer with an asterisk (*) at the beginning of the correct option.

Example:
Question 1: "question body?"
a. Option A
b. Option B
c. *Option C (Correct Answer)
d. Option D"""

  prompt = PromptTemplate(template=template, input_variables=["paragraph"])

  llm_chain = LLMChain(prompt=prompt, llm=llm)

  paragraph = input_data.paragraph

  response = llm_chain.run(paragraph)
  index = response.find('Question')
  res = response[index:]
  for responses in response.split('\n'):
    if 'Question' in responses or all(option in responses for option in ['a', 'b', 'c', 'd']):
      res += responses

  return res

@app.get("/generate-topics/")
def generate_topics(input_data: ParagraphInput):
  template = """
  You are an intelligent chatbot. Provide the following paragraph with a topic name.
  Paragraph: {paragraph}
  Topic:"""

  prompt = PromptTemplate(template=template, input_variables=["paragraph"])

  llm_chain = LLMChain(prompt=prompt, llm=llm)

  paragraph = input_data.paragraph

  response = llm_chain.run(paragraph)
  # print(response)
  for responses in response.split('\n'):
    if 'Topic' in responses:
      topic = responses
      # print(topic)
      return topic



In [ ]:
# Run the FastAPI server using uvicorn
token = '2eLEdRozNxC9uUlmWfhs1L6Lzjn_37pnGUm35Y2Z2DTsRYrmT'
ngrok.set_auth_token(token)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Exception in thread Thread-14 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


Public URL: https://fe9b-35-247-137-1.ngrok-free.app


INFO:     Started server process [3095]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


INFO:     58.65.135.186:0 - "GET /generate-mcqs/ HTTP/1.1" 200 OK
